#################### ONLY RUN 50 STOCKS AT A TIME OR LOWER THE PAGES TO 10><br><br>
This script is to perform web scraping on Google search (news tab). <br>

In [41]:
# Goal is to parse searches Google.com (news) for the stock symbols/company names.
# Default web searching URL is as follows:
    # https://www.google.com/search?q=COMPANY_NAME&tbm=nws&start=PAGE_NUMBER
        # Replace "COMPANY_NAME" with search term and "PAGE_NUMER" with 0 = page 1, 10 = page 2, 20 = page 3, etc.
# URL/REST parameters/search options: https://developers.google.com/custom-search/v1/cse/list 
## Google API for Python is available: https://github.com/googleapis/google-api-python-client 

import pandas, json, numpy, requests, os, datetime, pytz, tweepy, sqlite3, time, re, random
from bs4 import BeautifulSoup

# BeautifulSoup Doc: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

#### Following the same first steps from the Times API script:
import pandas, json, numpy, requests, os, datetime, pytz, tweepy, sqlite3, time, re

NYSE_csv = pandas.read_csv('NYSE.txt', sep="\t", header=0).set_index('Symbol')
NYSE_csv.to_csv('NYSE_csv.csv')

AMEX_csv = pandas.read_csv('AMEX.txt', sep="\t", header=0).set_index('Symbol')
AMEX_csv.to_csv('AMEX_csv.csv')

stock_exchange_ticks_and_names = pandas.merge(NYSE_csv.reset_index(), AMEX_csv.reset_index(), how='outer')
stock_exchange_ticks_and_names.to_csv('merged_NYSE_AMEX.csv')
stock_exchange_ticks_and_names_copy = stock_exchange_ticks_and_names.copy().dropna()

regex1 = re.compile('[@_!#$%^&*()<>?/\|}{~:[\].]')
regex2 = re.compile('Cl ')

stock_exchange_ticks_and_names_removed = pandas.DataFrame()

for x, y in stock_exchange_ticks_and_names_copy.iterrows():

    if bool(regex1.search(y['Description'])) == False and bool(regex2.search(y['Description'])) == False and bool(regex1.search(y['Symbol'])) == False:
        stock_exchange_ticks_and_names_removed.loc[x, 'Symbol'] = y['Symbol']
        stock_exchange_ticks_and_names_removed.loc[x, 'Description'] = y['Description']

stock_exchange_ticks_and_names_removed = stock_exchange_ticks_and_names_removed.set_index('Symbol')
stock_exchange_ticks_and_names_removed = stock_exchange_ticks_and_names_removed.reset_index()
stock_exchange_ticks_and_names_removed.to_csv('merged_NYSE_AMEX_removed_google_news.csv')

### To account for the Times API daily limit and time of running the script, each list is limited to 200: 
stock_exchange_ticks_and_names_0_399 = stock_exchange_ticks_and_names_removed.iloc[0:400, :]
stock_exchange_ticks_and_names_400_799 = stock_exchange_ticks_and_names_removed.iloc[400:800, :]
stock_exchange_ticks_and_names_800_1199 = stock_exchange_ticks_and_names_removed.iloc[800:1200, :]
stock_exchange_ticks_and_names_1200_1599 = stock_exchange_ticks_and_names_removed.iloc[1200:1600, :]
stock_exchange_ticks_and_names_1600_1999 = stock_exchange_ticks_and_names_removed.iloc[1600:2000, :]
stock_exchange_ticks_and_names_2000_2399 = stock_exchange_ticks_and_names_removed.iloc[2000:2400, :]
stock_exchange_ticks_and_names_2400_2799 = stock_exchange_ticks_and_names_removed.iloc[2400:2800, :]
stock_exchange_ticks_and_names_2800_3199 = stock_exchange_ticks_and_names_removed.iloc[2800:3200, :]
stock_exchange_ticks_and_names_3200_3599 = stock_exchange_ticks_and_names_removed.iloc[3200:3600, :]
stock_exchange_ticks_and_names_3600_3999 = stock_exchange_ticks_and_names_removed.iloc[3600:4000, :]
stock_exchange_ticks_and_names_4000_4399 = stock_exchange_ticks_and_names_removed.iloc[4000:4400, :]
stock_exchange_ticks_and_names_4400_4799 = stock_exchange_ticks_and_names_removed.iloc[4400:4800, :]
stock_exchange_ticks_and_names_4800_4880 = stock_exchange_ticks_and_names_removed.iloc[4800:, :]

stock_indices_df = pandas.DataFrame({'Description': ['S%26P', 'Dow', "Nasdaq"], 'Symbol': ['.INX', '.DJI', ".IXIC"]})


## Google Search Parsing Starts: _____________________________________________________________________________

def google_search_news(search_term, pages=10):
    pages = list(numpy.arange(0, pages*10, 10))
    filepath = os.getcwd() + '\\Google Search - News\\'

    for x in search_term.Description:
        print(x) ### This is to indicate where a error might have occurred.

        dictionary_all = {}
        master_df = pandas.DataFrame()
        headline_list = []
        web_url_list = []
        source_list = []
        time_list = []
        snippet_list = []

        for y in pages:
            get_URL = "https://www.google.com/search?q=" + x + "&tbm=nws&start=" + str(y) 
            Google_requests = requests.get(get_URL)

            soup = BeautifulSoup(Google_requests.text, 'html.parser')
            
            # Headline Parsing: ----------------------------------------------------------------------------
            temp_list = list(soup.find_all("div", class_ ="BNeawe vvjwJb AP7Wnd"))
            

            for u in temp_list:
                string = str(u)
                result = re.search('<div class="BNeawe vvjwJb AP7Wnd">(.*)</div>', string)
                headline_list.append(result.group(1))

            # Source Parsing: -------------------------------------------------------------------------------
            temp_list = list(soup.find_all("div", class_ ="BNeawe UPmit AP7Wnd"))

            for u in temp_list:
                string = str(u)
                result = re.search('<div class="BNeawe UPmit AP7Wnd">(.*)</div>', string)
                if result is None:
                    no_source = ''
                    source_list.append(no_source)
                else:
                    source_list.append(result.group(1))
                
            # Snippet: ------------------------------------------------------------------------------------            
            temp_list = list(soup.find_all("div", class_ ="BNeawe s3v9rd AP7Wnd"))

            for u in temp_list:
                string = str(u)
                if bool(re.search('<span class="r0bn4c rQMQod"> · </span>(.*)</div></div></div>', string)) == True:
                    result = re.search('<span class="r0bn4c rQMQod"> · </span>(.*)</div></div></div>', string)
                    snippet_list.append(result.group(1))

            # Date Posted (approximate): ------------------------------------------------------------------
            temp_list = list(soup.find_all("span", class_ ="r0bn4c rQMQod"))
            current_time = datetime.datetime.now()

            for u in temp_list:
                string = str(u)
                if bool(re.search('<span class="r0bn4c rQMQod">(.*)ago</span>', string)) == True:
                    result = re.search('<span class="r0bn4c rQMQod">(.*)ago</span>', string)
                    
                    digits = int(re.search('\d*', result.group(1)).group(0))
                    
                    if 'min' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                        
                    elif 'hour' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                    
                    elif 'day' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                        
                    elif 'week' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                        
                    elif 'month' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                        
                    elif 'year' in result.group(1):
                        diff_time = datetime.timedelta(digits)
                    
                    past_time = current_time - diff_time
                    
                    time_list.append(past_time.isoformat())

        # Merging all lists into a dictionary. Then the dictionary into a DataFrame:
        print(len(headline_list))
        print(len(snippet_list))
        print(len(time_list))
        print(len(source_list))
        dictionary_all = {'headline': headline_list, 'snippet': snippet_list, 'pub_date': time_list, 'source': source_list}
        combined_df = pandas.DataFrame(dictionary_all)
        combined_df['Company'] = x
        
        if not os.path.exists(filepath):
            os.makedirs(filepath)

        # Same file saving code as in Times API script (except duplicate matching is done on headline instead of date):
        # Matching is done on headline because it will eliminate "reblogged" articles.
        if os.path.exists(filepath + x + '.csv') == True:
            temp_df = pandas.read_csv(filepath + x + '.csv')
            print('2')
            #### This should cover the situation where pages != 1
            master_df = pandas.concat([temp_df, combined_df], ignore_index=True).drop_duplicates('headline').sort_values(by='pub_date', ascending=False)
            print('3')
            master_df.to_csv(filepath + x + '.csv')
        print('7')
        if os.path.exists(filepath + x + '.csv') == False:
            print('6')
            combined_df.to_csv(filepath + x + '.csv')

In [ ]:
google_search_news(stock_indices_df) ###

google_search_news(stock_exchange_ticks_and_names_0_399)  ### 
google_search_news(stock_exchange_ticks_and_names_400_799)  ### 
google_search_news(stock_exchange_ticks_and_names_800_1199)  ### 
google_search_news(stock_exchange_ticks_and_names_1200_1599)  ### 
google_search_news(stock_exchange_ticks_and_names_1600_1999)  ### 
google_search_news(stock_exchange_ticks_and_names_2000_2399)  ### 
google_search_news(stock_exchange_ticks_and_names_2400_2799)  ### 
google_search_news(stock_exchange_ticks_and_names_2800_3199)  ###
google_search_news(stock_exchange_ticks_and_names_3200_3599)  ###
google_search_news(stock_exchange_ticks_and_names_3600_3999)  ###
google_search_news(stock_exchange_ticks_and_names_4000_4399)  ### 
google_search_news(stock_exchange_ticks_and_names_4400_4799)  ###
google_search_news(stock_exchange_ticks_and_names_4800_4880)  ###